### Import

In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

### GPU

In [3]:
torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [4]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono'
excl_folders = ['NoFX_mono', 'NoFX_mono_preprocessed']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=6

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [5]:
len(dataset)

163488

### TRAIN MultiNET

In [6]:
# model
net = models.MultiNet().to(device)
# optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
# loss function
loss_func_fx = nn.CrossEntropyLoss()
loss_func_set = nn.L1Loss()

In [7]:
print(net)

MultiNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1_a): Linear(in_features=6264, out_features=120, bias=True)
  (fc2_a): Linear(in_features=120, out_features=60, bias=True)
  (out_a): Linear(in_features=60, out_features=14, bias=True)
  (fc1_b): Linear(in_features=6264, out_features=120, bias=True)
  (fc2_b): Linear(in_features=120, out_features=60, bias=True)
  (out_b): Linear(in_features=60, out_features=6, bias=True)
)


In [8]:
# TRAIN and TEST MultiNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

start = time.time()

for epoch in range(50):
    train_loss, train_correct, train_results = trainer.train_multi_net(
        model=net,
        optimizer=optimizer, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_multi_net(
        model=net,
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_multi_net(
        model=net,
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        loss_function_fx=loss_func_fx, 
        loss_function_set=loss_func_set,
        device=device
    )
    
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

stop = time.time()
print(f"Training time: {stop - start}s")

00/117711 (96%)]	Total Loss: 185.7705	Avg Loss: 0.0016
Train Epoch: 47	[114000/117711 (97%)]	Total Loss: 187.5006	Avg Loss: 0.0016
Train Epoch: 47	[115000/117711 (98%)]	Total Loss: 189.1175	Avg Loss: 0.0016
Train Epoch: 47	[116000/117711 (98%)]	Total Loss: 190.7027	Avg Loss: 0.0016
Train Epoch: 47	[117000/117711 (99%)]	Total Loss: 192.3943	Avg Loss: 0.0016
====> Epoch: 47	Total Loss: 193.5080	 Avg Loss: 0.0016	 Fx Loss: 153.6135	 Set Loss: 39.8945
		Correct: 57954/117711	Fx Correct: 107643/117711	Set Correct: 62188/117711
		Percentage Correct: 49.23	Percentage Fx Correct: 91.45	Percentage Set Correct: 52.83
====> Val Loss: 29.0056	 Avg Loss: 0.0022	 Fx Loss: 23.7400	 Set Loss: 5.2656
		Correct: 5510/13079	Fx Correct: 11851/13079	Set Correct: 5951/13079
		Percentage Correct: 42.13	Percentage Fx Correct: 90.61	Percentage Set Correct: 45.50
====> Test Loss: 73.0557	 Avg Loss: 0.0022	 Fx Loss: 59.9561	 Set Loss: 13.0996
		Correct: 13796/32698	Fx Correct: 29516/32698	Set Correct: 14972/3269

In [9]:
# SAVE
models_folder = 'saved_models'
model_name = '20200908_multinet'
results_folder = 'saved_results'

In [10]:
# SAVE MODEL
torch.save(net, '%s/%s' % (models_folder, model_name))

In [11]:
# SAVE RESULTS
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
all_train_results_npy = np.array(all_train_results)

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
all_val_results_npy = np.array(all_val_results)

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
all_test_results_npy = np.array(all_test_results)

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_losses')), arr=all_train_losses_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_correct')), arr=all_train_correct_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_train_results')), arr=all_train_results_npy, allow_pickle=True)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_losses')), arr=all_val_losses_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_correct')), arr=all_val_correct_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_val_results')), arr=all_val_results_npy, allow_pickle=True)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_losses')), arr=all_test_losses_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_correct')), arr=all_test_correct_npy, allow_pickle=True)
np.save(file=('%s/%s/%s' % (results_folder, model_name, 'all_test_results')), arr=all_test_results_npy, allow_pickle=True)

np.save(file=('%s/%s/%s' % (results_folder, model_name, 'fx_labels')), arr=fx_labels_npy, allow_pickle=True)